# Create Public School Machine Learning Datasets
** This program creates all the _ML datasets in the NCEA repository.** 
* This notebook reads each School Dataset file located at \EducationDataNC\ *schoolYear* \School Datasets\ as input data.
* Different school years are processed by changing the *schoolYear* parameter.
* Different input / output files are processed / created by changing the *inputFileName* paramter in the cell below.  
* While a single program is used to create all the _ML datasets, one program copy per dataset is maintained in the repositiory so the dataset specific tranformation reports may be reviewed. 

**Datasets ending in ML are preprocessed for Machine Learning and go through the following transformations: **
1. Missing student body racial compositions are imputed using district averages.
2. Columns that have the same value in every single row are deleted.
3. Columns that have a unique value in every single row (all values are different) are deleted.
4. Empty columns (all values are NA or NULL) are deleted.
5. Numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
6. Remaining numeric, non-race columns with missing values are imputed / populated with 0.  In many cases, schools are not reporting values when they are zero. However, mean imputation or some other more sophisticated strategy might be considered here.
7. Categorical / text based columns with > *uniqueThreshold* unique values are deleted.
8. All remaining categorical / text based columns are one-hot encoded.  In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
9. Duplicated or highly similar columns with > 95% correlation are delelted.    

In [151]:
#import required Libraries
import pandas as pd
import numpy as np
import os
import string

#**********************************************************************************
# Set the following variables before running this code!!!
#**********************************************************************************
#All raw data files are processed for the year below
#Alter the Years Here: 2014-2019
schoolYear = 2019

#Location where copies of the raw data files will be read in from csv files.
#Your Path Here
dataDir = 'C:/Users/taniat470s/Desktop/SMU_course/DS7331/EduData/EducationDataNC/' + str(schoolYear) + '/School Datasets/'

#Name of the file to be processed
inputFileName = 'PublicSchools' + str(schoolYear)
#inputFileName = 'PublicHighSchools' + str(schoolYear)
#inputFileName = 'PublicMiddleSchools' + str(schoolYear)
#inputFileName = 'PublicElementarySchools' + str(schoolYear)

#Input file being transformed for machine learning 
inputFile = dataDir + inputFileName + '.csv'

#Location where the new school datasets will be created.
#Your Path Here
outputDir = 'C:/Users/taniat470s/Desktop/SMU_course/DS7331/EduData/EducationDataNC/' + str(schoolYear) + '/Machine Learning Datasets/'

#Missing Data Threshold (Per Column)
missingThreshold = 0.60

#Unique Value Threshold (Per Column)
#Delete Columns >  uniqueThreshold unique values prior to one-hot encoding. 
#(each unique value becomes a new column during one-hot encoding)
uniqueThreshold = 25

#Read in the School Data File
#schoolData = pd.read_csv(inputFile, low_memory=False, dtype={'unit_code': object})
schoolData = pd.read_csv(inputFile, low_memory=False, dtype={'agency_code': object})
print('*********Start: Beginning Column and Row Counts********************************************')
schoolData.info(verbose=False)

#Select only public schools as charter schools are missing data for many columns.
#schoolData = schoolData[(schoolData['type_cd'] == 'P') & (schoolData['student_num'] > 0)]
#2018
#schoolData = schoolData[(schoolData['designation_type'] == 'P') & (schoolData['avg_student_num'] > 0)]
#2019
schoolData = schoolData[(schoolData['designation_type'] == 'P')] #& (schoolData['avg_student_num'] > 0)] -rcd_admin missing

print('\r\n*********After: Selecting Only Public School Campuses**********************************')
schoolData.info(verbose=False)

#Save primary key
#2014-2017
#unit_code = schoolData['unit_code']
#2018,2019
agency_code = schoolData['agency_code']
#Convert zip code to string
#2014-2017
#schoolData['szip_ad'] = schoolData['szip_ad'].astype('object')
#2018,2019
schoolData['zip'] = schoolData['zip'].astype('object')
#2014-2016
#Rename Lea_Name to District Name
#schoolData.rename(columns={'Lea_Name': 'District Name'}, inplace=True)

*********Start: Beginning Column and Row Counts********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Columns: 1485 entries, agency_code to welcome_url
dtypes: float64(1435), object(50)
memory usage: 30.7+ MB

*********After: Selecting Only Public School Campuses**********************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 1485 entries, agency_code to welcome_url
dtypes: float64(1435), object(50)
memory usage: 28.5+ MB


# Prepare Consolidated Dataset for Machine Learning
**Below we perform operations on the entire dataset to remove columns and update row values that could cause problems during machine learning.**

## Student Body Racial Composition Features 
**Impute / update missing Student Body Racial Composition Fields using mean imputation.**
* When there are no racial composition percentages for a particular school campus / unit_code, fill in the missing values 

In [152]:
schoolData.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Data columns (total 1485 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    agency_code                          object 
 1    category_code                        object 
 2    agency_level                         object 
 3    lea_code                             object 
 4    designation_type                     object 
 5    name                                 object 
 6    county                               object 
 7    street_addr                          object 
 8    stree_addr2                          object 
 9    city                                 object 
 10   state                                object 
 11   zip                                  object 
 12   phone                                object 
 13   grade_span                           object 
 14   school_type                          object 
 15   calendar_type     

In [154]:
#Get Student Body Racial Composition Fields
raceCompositionFields = schoolData.filter(regex='Indian|Asian|Hispanic|Black|White|PacificIsland|TwoOrMore|Minority')\
                                  .filter(regex='Pct').columns
    
rowsBefore = schoolData[raceCompositionFields].isnull().T.any().T.sum()

#Update missing race values with the district average when avaiable (No district averages for charter schools) 
#schoolData[raceCompositionFields] = schoolData.groupby('District Name')[raceCompositionFields]\
#                                              .transform(lambda x: x.fillna(x.mean()))
schoolData[raceCompositionFields] = schoolData.groupby('lea_code')[raceCompositionFields]\
                                              .transform(lambda x: x.fillna(x.mean()))

    #Review dataset contents after Racial Composition Imputation
print('*********After: Updating Missing Racial Compostion Values****************************')   
rowsAfter = schoolData[raceCompositionFields].isnull().T.any().T.sum()
rowsUpdated = rowsBefore - rowsAfter
print('Rows Updated / Imputed: ', rowsUpdated) 
print('\r\nTotal Rows Missing Racial Compositions By District Name') 
#schoolData['District Name'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()
#2018,2019
schoolData['lea_code'][schoolData[raceCompositionFields].isnull().T.any().T].value_counts()

*********After: Updating Missing Racial Compostion Values****************************
Rows Updated / Imputed:  0

Total Rows Missing Racial Compositions By District Name


010LEA    36
070LEA    15
090LEA    13
020LEA    11
060LEA    11
040LEA    10
080LEA     7
050LEA     6
030LEA     4
Name: lea_code, dtype: int64

## Remove Columns with Problematic Data
**Here we remove entire columns that could cause problems during machine learning.  The following operations are performed:**
* Remove any columns that have the same value in every single row.
* Remove any columns that have a unique value in every single row (all values are different).
* Remove empty columns (all values are NA or NULL).

In [155]:
#Remove any fields that have the same value in all rows
UniqueValueCounts = schoolData.nunique(dropna=False)
SingleValueCols = UniqueValueCounts[UniqueValueCounts == 1].index
schoolData = schoolData.drop(SingleValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with the same value in every row.*******************')
schoolData.info(verbose=False)
print('\r\nColumns Deleted: ', len(SingleValueCols))

*********After: Removing columns with the same value in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 753 entries, agency_code to welcome_url
dtypes: float64(709), object(44)
memory usage: 14.5+ MB

Columns Deleted:  732


In [156]:
#Remove any fields that have unique values in every row
schoolDataRecordCt = schoolData.shape[0]
UniqueValueCounts = schoolData.apply(pd.Series.nunique)
AllUniqueValueCols = UniqueValueCounts[UniqueValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(AllUniqueValueCols, axis=1)

#Review dataset contents after drops
print('*********After: Removing columns with unique values in every row.*******************')
schoolData.info(verbose=False)
print('\r\nColumns Deleted: ', len(AllUniqueValueCols))

*********After: Removing columns with unique values in every row.*******************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 753 entries, agency_code to welcome_url
dtypes: float64(709), object(44)
memory usage: 14.5+ MB

Columns Deleted:  0


In [157]:
#Remove any empty fields (null values in every row)
schoolDataRecordCt = schoolData.shape[0]
NullValueCounts = schoolData.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts == schoolDataRecordCt].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with null / blank values in every row.*************')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with null / blank values in every row.*************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 753 entries, agency_code to welcome_url
dtypes: float64(709), object(44)
memory usage: 14.5+ MB

Columns Deleted:  0


## Handle Other Missing Values Types
* Here we eliminate any numeric columns with more than the percentage of missing values specified by the *missingThreshold* parameter.
* All remaining non-race, numeric column missing values are populated with 0.
* In many cases, it seems that schools are not simply not reporting values when they are zero. However, mean imputation or some other strategy might be considered.

In [158]:
#Isolate continuous and categorical data types
#These are indexers into the schoolData dataframe and may be used similar to the schoolData dataframe 
sD_boolean = schoolData.loc[:, (schoolData.dtypes == bool) ]
sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]
sD_continuous = schoolData.loc[:, (schoolData.dtypes != bool) & (schoolData.dtypes != object)]
print ("Boolean Columns: ", sD_boolean.shape[1])
print ("Nominal Columns: ", sD_nominal.shape[1])
print ("Continuous Columns: ", sD_continuous.shape[1])
print ("Columns Accounted for: ", sD_nominal.shape[1] + sD_continuous.shape[1] + sD_boolean.shape[1])

Boolean Columns:  0
Nominal Columns:  44
Continuous Columns:  709
Columns Accounted for:  753


In [159]:
#Eliminate continuous columns with more than missingThreshold percentage of missing values
schoolDataRecordCt = sD_continuous.shape[0]
missingValueLimit = schoolDataRecordCt * missingThreshold
NullValueCounts = sD_continuous.isnull().sum()
NullValueCols = NullValueCounts[NullValueCounts >= missingValueLimit].index
schoolData = schoolData.drop(NullValueCols, axis=1)

#Review dataset contents after empty field drops
print('*********After: Removing columns with >= missingThreshold % of missing values******')
schoolData.info(verbose=False)
print ('\r\nColumns Deleted: ', len(NullValueCols))

*********After: Removing columns with >= missingThreshold % of missing values******
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 252 entries, agency_code to welcome_url
dtypes: float64(208), object(44)
memory usage: 4.9+ MB

Columns Deleted:  501


## One-Hot Encoding of Categorical Variables
**All categorical / string variables are converted to numberic variables via one hot encoding.  Each unique row value will become a new binary / numeric column in the dataset.**
* All remaining categorical columns are one-hot encoded.  
* In categorical columns, one-hot encoding creates one new boolean / binary field per unique value in the target column, converting all categorical columns to a numeric data type. 
* Prior to one-hot encoding, columns with > *uniqueThreshold* unique values are deleted.  

In [160]:
# #Delete categorical columns with > 25 unique values (Each unique value becomes a column during one-hot encoding)
# oneHotUniqueValueCounts = schoolData[sD_nominal.columns].apply(lambda x: x.nunique())
# oneHotUniqueValueCols = oneHotUniqueValueCounts[oneHotUniqueValueCounts >= uniqueThreshold].index
# schoolData.drop(oneHotUniqueValueCols, axis=1, inplace=True) 

# #Review dataset contents one hot high unique value drops
# print('*********After: Removing columns with >= uniqueThreshold unique values***********')
# schoolData.info(verbose=False)
# print '\r\nColumns Deleted: ', len(oneHotUniqueValueCols)

In [161]:
# #Isolate remaining categorical variables
# begColumnCt = len(schoolData.columns)
# sD_nominal = schoolData.loc[:, (schoolData.dtypes == object)]

# #one hot encode categorical variables
# schoolData = pd.get_dummies(data=schoolData, 
#                        columns=sD_nominal, drop_first=True)

# #Determine change in column count
# endColumnCt = len(schoolData.columns)
# columnsAdded = endColumnCt - begColumnCt

# #Review dataset contents one hot high unique value drops
# print 'Columns To One-Hot Encode: ', len(sD_nominal.columns)
# print('\r\n*********After: Adding New Columns Via One-Hot Encoding*************************')
# schoolData.info(verbose=False)
# print '\r\nNew Columns Created Via One-Hot Encoding: ', columnsAdded

## Impute any Remaining Missing Values as Zero

In [162]:
#Print out all the missing value rows
pd.set_option('display.max_rows', 1000)

print('\r\n*********The Remaining Missing Values Below will be set to Zero!*************************')

#Check for Missing values 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values




*********The Remaining Missing Values Below will be set to Zero!*************************


,Variable Name,Number Missing Values
6,stree_addr2,2504
9,phone,3
12,calendar_type,1
13,url,71
14,title_I,1110
15,stem,2488
16,School Name,124
17,IndianPct,113
18,AsianPct,113
19,HispanicPct,113


In [163]:
#Replace all remaining NaN with 0
schoolData = schoolData.fillna(0)

#Check for Missing values after final imputation 
missing_values = schoolData.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values

,Variable Name,Number Missing Values


## Identify and Remove Highly Correlated Features
**Find and remove any columns / features that are > 95% correlated**
* https://stackoverflow.com/questions/39409866/correlation-heatmap
* https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
* https://codeyarns.com/2015/04/20/how-to-change-font-size-in-seaborn/

In [164]:
# calculate the correlation matrix
corr_matrix  = schoolData.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [165]:
#Get all of the correlation values > 95%
x = np.where(upper > 0.95)

#Display all field combinations with > 95% correlation
cf = pd.DataFrame()
cf['Field1'] = upper.columns[x[1]]
cf['Field2'] = upper.index[x[0]]

#Get the correlation values for every field combination. (There must be a more pythonic way to do this!)
corr = [0] * len(cf)
for i in range(0, len(cf)):
    corr[i] =  upper[cf['Field1'][i]][cf['Field2'][i]] 
    
cf['Correlation'] = corr

print ('There are ', str(len(cf['Field1'])), ' field correlations > 95%.')
cf

There are  68  field correlations > 95%.


,Field1,Field2,Correlation
0,IndianMalePct,IndianPct,0.992748
1,IndianFemalePct,IndianPct,0.992154
2,AsianMalePct,AsianPct,0.969875
3,AsianFemalePct,AsianPct,0.971541
4,HispanicMalePct,HispanicPct,0.964099
5,HispanicFemalePct,HispanicPct,0.963415
6,BlackMalePct,BlackPct,0.972500
7,BlackFemalePct,BlackPct,0.968160
8,WhiteMalePct,WhitePct,0.980191
9,WhiteFemalePct,WhitePct,0.977463


In [166]:
print ('Dropping the following ', str(len(to_drop)), ' highly correlated fields.')
to_drop

Dropping the following  56  highly correlated fields.


['IndianMalePct',
 'AsianMalePct',
 'HispanicMalePct',
 'BlackMalePct',
 'WhiteMalePct',
 'MinorityMalePct',
 'IndianFemalePct',
 'AsianFemalePct',
 'HispanicFemalePct',
 'BlackFemalePct',
 'WhiteFemalePct',
 'MinorityFemalePct',
 'eg_index_RD_ALL',
 'eg_score_RD_ALL',
 'pct_ELS_ELP',
 'pct_met_Reading Grades 3-8_PART',
 'pct_Math Grades 3-8_ALL_PART_DET',
 'pct_Reading Grades 3-8_ALL_PART_DET',
 'pct_Reading Grades 3-8_BL7_PART_DET',
 'pct_Reading Grades 3-8_EDS_PART_DET',
 'pct_Reading Grades 3-8_HI7_PART_DET',
 'pct_Reading Grades 3-8_SWD_PART_DET',
 'pct_Reading Grades 3-8_WH7_PART_DET',
 'pct_Science Grades 5&8_ALL_PART_DET',
 'rd_score',
 'spg_score',
 'ma_spg_score',
 'rd_spg_score',
 'ach_score_BL7_SPG2',
 'ach_score_ELS_SPG2',
 'ach_score_HI7_SPG2',
 'ach_score_WH7_SPG2',
 'eg_score_BL7_SPG2',
 'eg_score_EDS_SPG2',
 'eg_score_ELS_SPG2',
 'eg_score_HI7_SPG2',
 'eg_score_SWD_SPG2',
 'eg_score_WH7_SPG2',
 'scgs_score_WH7_SPG2',
 'spg_score_BL7_SPG2',
 'spg_score_ELS_SPG2',
 'spg_

In [167]:
#Check columns before drop 
print('\r\n*********Before: Dropping Highly Correlated Fields*************************************')
schoolData.info(verbose=False)

# Drop the highly correlated features from our training data 
schoolData = schoolData.drop(to_drop, axis=1)

#Check columns after drop 
print('\r\n*********After: Dropping Highly Correlated Fields**************************************')
schoolData.info(verbose=False)


*********Before: Dropping Highly Correlated Fields*************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 252 entries, agency_code to welcome_url
dtypes: float64(208), object(44)
memory usage: 4.9+ MB

*********After: Dropping Highly Correlated Fields**************************************
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Columns: 196 entries, agency_code to welcome_url
dtypes: float64(152), object(44)
memory usage: 3.8+ MB


In [168]:
#Restore the unit_code before saving
schoolData['unit_code'] = unit_code
#Save the final dataset to a .csv file
schoolData.to_csv(outputDir + inputFileName + '_MLnew.csv', sep=',', index=False)

In [169]:
print('*********FINAL DATASET DETAILS*********************************************************\r\n')
schoolData.info(verbose=True)

*********FINAL DATASET DETAILS*********************************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 2 to 2706
Data columns (total 197 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   agency_code                          object 
 1   category_code                        object 
 2   lea_code                             object 
 3   name                                 object 
 4   county                               object 
 5   street_addr                          object 
 6   stree_addr2                          object 
 7   city                                 object 
 8   zip                                  object 
 9   phone                                object 
 10  grade_span                           object 
 11  school_type                          object 
 12  calendar_type                        object 
 13  url                                  object 
 14

In [170]:
import sklearn
import pandas as pd

print('Sklearn Version: ' + sklearn.__version__)
print('Pandas Version: ' + pd.__version__)

Sklearn Version: 0.22.1
Pandas Version: 1.0.3


In [171]:
print ('Output File Location:\r\n\r\n' + outputDir + inputFileName + '_MLnew.csv')

Output File Location:

C:/Users/taniat470s/Desktop/SMU_course/DS7331/EduData/EducationDataNC/2019/Machine Learning Datasets/PublicSchools2019_MLnew.csv


In [172]:
schoolData.shape

(2517, 197)